In [ ]:
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import cv2



In [ ]:
def load_data(data_path):
    images=[]
    ages=[]
    for filename in os.listdir(data_path):
        try:
            age=int(filename.split("_")[0])
            img=Image.open(os.path.join(data_path, filename)).resize((100, 100))
            img=np.array(img)/255.0
            if img.shape==(100,100,3):
                images.append(img)
                ages.append(age)

        except:
            continue
    return np.array(images), np.array(ages)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Age Detection/Age Detection/UTKFace.zip'  # Make sure this matches the uploaded file name

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Age Detection/Age Detection/UTKFace')  # Extract into a folder named "UTKFace"


In [ ]:
print(os.getcwd())           # Shows your current working directory
print(len(os.listdir("drive/MyDrive/Age Detection/Age Detection/UTKFace/UTKFace")))


/content
21807


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

class UTKFaceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size=32, img_size=(100, 100), shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_paths = [self.image_paths[i] for i in batch_indexes]
        batch_labels = [self.labels[i] for i in batch_indexes]

        batch_images = []
        for path in batch_paths:
            img = cv2.imread(path)
            img = cv2.resize(img, self.img_size)
            img = img / 255.0
            batch_images.append(img)

        return np.array(batch_images), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [ ]:
import glob

dataset_path = "drive/MyDrive/Age Detection/Age Detection/UTKFace/UTKFace"
all_files = glob.glob(os.path.join(dataset_path, "*.jpg"))  # or .jpeg/.png if needed

# Parse age from filename
all_ages = [int(os.path.basename(f).split("_")[0]) for f in all_files]

# Split dataset
from sklearn.model_selection import train_test_split
train_paths, val_paths, train_ages, val_ages = train_test_split(all_files, all_ages, test_size=0.2)


In [ ]:
train_gen = UTKFaceDataGenerator(train_paths, train_ages, batch_size=32)
val_gen = UTKFaceDataGenerator(val_paths, val_ages, batch_size=32)


In [ ]:
images,ages=load_data('drive/MyDrive/Age Detection/Age Detection/UTKFace/UTKFace')

In [ ]:
def build_model():
    model = models.Sequential([
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3)),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64,(3,3),activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128,(3,3),activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.Flatten(),

        layers.Dense(128, activation='relu'),
        layers.Dense(1)


    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [ ]:
def train_model():
    # X_train,X_test,Y_train,Y_test=train_test_split(images,ages,test_size=0.2)
    model=build_model()

    print(type(model))


    model.fit(train_gen, validation_data=val_gen, epochs=60)
    model.save('/content/drive/MyDrive/new_age_model.h5')
    print("Model saved to age_model.h5")
    return model

In [ ]:
train_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<class 'keras.src.models.sequential.Sequential'>


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
  6/545 ━━━━━━━━━━━━━━━━━━━━ 1:16:56 9s/step - loss: 821.6451 - mae: 24.7120

In [ ]:
def range_age(age):

    if 0 <= age <= 2:
        return "0-2"
    elif 3 <= age <= 9:
        return "3-9"
    elif 10 <= age <= 20:
        return "10-20"
    elif 21 <= age <= 27:
        return "21-27"
    elif 28 <= age <= 45:
        return "28-45"
    elif 46 <= age <= 65:
        return "45-65"
    else:
        return "65+"



def live_age_prediction():
    if not os.path.exists("age_model.h5"):
        print("⚠️ Model not found. Please train the model first.")
        return

    model = tf.keras.models.load_model("age_model.h5")
    cap = cv2.VideoCapture(0)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (100, 100)) / 255.0
            face_input = np.expand_dims(face_resized, axis=0)

            age_pred = model.predict(face_input, verbose=0)[0][0]
            range=range_age(age_pred)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f'Age: {range}', (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        cv2.imshow("Live Age Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


live_age_prediction()

error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
